In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import networkx as nx

In [ ]:
# Command to use the full widht of the screen
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }/style>"))

In [ ]:
def BACN(init_nodes, final_nodes, m_param, l_param=0, CN=False, DIR=False, Initial_Connectivity='all'):
    assert init_nodes > 1
    assert init_nodes < final_nodes
    assert init_nodes >= m_param
    assert l_param >= 0

    #INITIAL NETWORK:
    A = np.zeros((final_nodes, final_nodes))
    if Initial_Connectivity == 'all': #all-to-all
        for i in range(init_nodes):
            for j in range(i+1, init_nodes):
                A[i,j] = A[j,i] = 1
    elif Initial_Connectivity == 'my_random': #i.e. "random"
        for i in range(init_nodes):
            for j in range(i+1, init_nodes):
                A[i,j] = A[j,i] = int(np.random.random()+0.5)
            print(i, "A[i]", A[i])
            if not A[i].any() != 0:
                if i+1 != init_nodes: #helpful if the last row is still full of zeroes
                    print("if")
                    rand_column = np.random.choice(range(i+1, init_nodes))
                    print(range(i+1, init_nodes))
                else:
                    print("else")
                    rand_column = np.random.choice(range(i))
                    print(range(i))
                print("rand_column", rand_column)
                A[i, rand_column] = A[rand_column, i] = 1
    elif Initial_Connectivity == 'regular_2':
        init_deg = 2
        G = nx.random_regular_graph(init_deg, init_nodes)
        for e in G.edges():
            A[e[0],e[1]] = A[e[1],e[0]] = 1
    
    #NETWORK EVOLUTION:
    for i in range(init_nodes, final_nodes):
        if DIR:
            p_deg_source = np.sum(A[:i,:], axis=1)
            p_deg_source /= np.sum(p_deg_source)
            p_deg_target = np.sum(A[:,:i], axis=0)
            p_deg_target /= np.sum(p_deg_target)
            rand_source = np.random.choice(i, size=m_param, p=p_deg_source, replace=False)
            A[rand_source, i] = 1
            rand_target = np.random.choice(i, size=m_param, p=p_deg_target, replace=False)
            A[i, rand_target] = 1
        else:
            p_deg = np.sum(A, axis=1)[:i]
            p_deg /= np.sum(p_deg)
            rand_node = np.random.choice(i, size=m_param, p=p_deg, replace=False)
            A[i, rand_node] = A[rand_node, i] = 1
        if CN and l_param > 0:
            S = A[:i+1,:i+1]
            cn_mat = S @ S
            S[np.eye(S.shape[0], dtype=bool)] = -1
            missing = np.nonzero(S==0)
            if DIR==False:
                #only upper triangle of adjacency matrix:
                missing = (missing[0][missing[0]<missing[1]], missing[1][missing[0]<missing[1]])
            if missing[0].size != 0:
                p_cn = cn_mat[missing]
                p_cn /= np.sum(p_cn)
                if np.isnan(p_cn).all():
                    if l_param < missing[0].size:
                        rand_edge = np.random.choice(missing[0].size, size=l_param, replace=False)
                    else:
                        rand_edge = range(missing[0].size)
                else:
                    if l_param > np.count_nonzero(p_cn):
                        rand_edge = np.random.choice(missing[0].size, size=np.count_nonzero(p_cn), p=p_cn, replace=False)
                    else:
                        rand_edge = np.random.choice(missing[0].size, size=l_param, p=p_cn, replace=False)
                a = missing[0][rand_edge]
                b = missing[1][rand_edge]
                if DIR:
                    A[a,b] =  1
                else:
                    A[a,b] = A[b,a] = 1
            S[np.eye(S.shape[0], dtype=bool)] = 0
    return A

In [ ]:
testBACN = BACN(2,6,1,0,CN=True,DIR=True,Initial_Connectivity='all')
testBACN

In [ ]:
np.array_equal(testBACN, testBACN.T)

In [ ]:
def cns_number(x, links):
    c_m = x@x
    cn = c_m[links[:,0], links[:,1]]
    return cn

In [ ]:
def get_links_to_del(x, fraction=0.1, loops=1, DIR=False):
    if DIR:
        present = np.argwhere(x)
    else:
        present = np.argwhere(np.triu(x, 1) != 0)#== 1)
    num_to_del = int(present.shape[0] * fraction)
    idx_to_del = [np.random.choice(present.shape[0], size=num_to_del, replace=False) for i in range(loops)]
    links_to_del = np.array([present[idx] for idx in idx_to_del])
    return links_to_del

In [ ]:
def delete_links(x, links_to_del, DIR=False):
    """Deletes the specified links from an adjacency matrix."""
    z = np.copy(x)
    for link in links_to_del:
        if DIR:
            z[link[0], link[1]] = 0
        else:
            z[link[0], link[1]] = z[link[1], link[0]] = 0
    return z

In [ ]:
def evaluate_predictions(true, pred, plot=False):
    precision, recall, _ = precision_recall_curve(true, pred)
    fpr, tpr, _ = roc_curve(true, pred)
    aupr = auc(recall, precision)
    auroc = auc(fpr, tpr)
    if plot:
        fig, ax = plt.subplots()
        ax.plot(recall, precision, 'go', label='Precision-Recall curve')
        ax.plot(fpr, tpr, '.', label='ROC curve')
        plt.legend()
        plt.show()
    return aupr, auroc

In [ ]:
def cross_val_from_notebook(adjacency, links_to_del, loops=1, DIR=False, raw_output=False, verbose=False):
    if raw_output:
        scores_list = []
        true_list = []
    else:
        results = np.zeros((loops, 2))
    for i in range(loops):
        #print("Loop:", i)
        if verbose:
            print('Trial {} of {}'.format(i+1, loops))
        x_ = delete_links(adjacency, links_to_del[i], DIR)
        missing_initial = np.argwhere(x_ == 0)
        #print("missing_initial", missing_initial, missing_initial.shape[0])
        missing_final = np.array([list(row) for row in missing_initial if row[0]!=row[1]]) #exclude the diagonal terms
        #print("missing_final", missing_final, missing_final.shape[0])
        if missing_final.shape[0] > 0:
            true_present = adjacency[missing_final[:,0], missing_final[:,1]]
            scores = cns_number(x_, missing_final)
            if False:#true_present.any():
                print("\n")
                #print(adjacency)
                #print(x_)
                #print("missing_initial:")
                #print(missing_initial, missing_initial.shape[0])
                #print("missing_final:")
                #print(missing_final, missing_final.shape[0])
                #print("true_present", true_present)
                #print("scores", scores)
            if raw_output:
                true_list.append(true_present)
                scores_list.append(scores)
            else:
                results[i] = evaluate_predictions(true_present, scores, plot=plot)
                #if not np.isnan(results[i]).any():
                #    print("results[i]", results[i])
        else:
            print("WTF?")
            break
    if raw_output:
        return np.array(true_list).flatten(), np.asarray(scores_list).flatten()
    else:
        #print("results")
        #print(results)
        return results

In [ ]:
def running_all_parameters(init_nodes, final_nodes, l, fraction, loops, CN, DIR, Initial_Connectivity):
    areas_under_curves = np.zeros((init_nodes+1, 2))
    for m in range(init_nodes+1):
        B = BACN(init_nodes, final_nodes, m, l, CN, DIR, Initial_Connectivity)
        links_to_del = get_links_to_del(B, fraction, loops, DIR)
        r1 = cross_val_from_notebook(B, links_to_del, loops, DIR)
        areas_under_curves[m,:] = np.mean(r1, axis=0)
    return areas_under_curves

In [ ]:
init_nodes=7
final_nodes=100
m=1
l=5
CN=True
DIR=False
fraction=0.1
loops=10
B = BACN(init_nodes, final_nodes, m, l, CN, DIR)
links_to_del = get_links_to_del(B, fraction, loops, DIR)
r1 = cross_val_from_notebook(B, links_to_del, loops, DIR)
print(np.mean(r1, axis=0))

In [ ]:
def ploting_all_parameters(initial, final, lmax, fraction, loops, CN, DIR, Initial_Connectivity):
    for i in range(initial, final):
        #print("\n")
        #print("initial number of nodes:", i)
        plt.figure(figsize=(20,3))
        for j in range(1, lmax+1):
            #print("CN-parameter:", j)
            a_u_c = running_all_parameters(i, final, j-1, fraction, loops, CN, DIR, Initial_Connectivity)
            plt.subplot(1,lmax,j)
            plt.title("I:{}, f:{}\nCN:{}".format(i, final, j-1))
            plt.xlabel("m")
            plt.ylim([0, 1])
            plt.plot(a_u_c[:,0], 'bx--', label='AUPR')
            plt.plot(a_u_c[:,1], 'ro--', label='AUROC')
            #plt.legend()
        plt.show()

In [ ]:
ploting_all_parameters(initial=30, final=35, lmax=10, fraction=0.1, loops=10, CN=True, DIR=False, Initial_Connectivity='regular_2')

In [ ]:
G = nx.random_regular_graph(2, 10)
G.edges()

In [ ]:
GG = np.array(nx.adjacency_matrix(G).todense())
GG

In [ ]:
nx.draw(G, pos=nx.spring_layout(G))

In [ ]:
[edge for edge in G.edges()]

In [ ]:
for e in G.edges():
    print(e)

In [ ]:
init_deg = 2
init_nodes = 5
final_nodes = 7
AA = np.zeros((final_nodes, final_nodes))
print(AA)
G = nx.random_regular_graph(init_deg, init_nodes)
for e in G.edges():
    AA[e[0],e[1]] = AA[e[1],e[0]] = 1
print(AA)